In [1]:
%load_ext autoreload
%autoreload 2

In [ ]:
from bench_lib.evaluation import benchmark_results_folder

rfolder = benchmark_results_folder()
folders = [
    # "gemma-3-4b-it",
    # "gemma-3-12b-it",
    # "gemma-3-27b-it_00",
    "qwen-2.5-vl",
    "gemini-2.5-pro-noschema",
]
files = [str(rfolder / f / "toxicainment_videos_log.jsonl") for f in folders]

In [3]:
import duckdb

con = duckdb.connect()
query = f"SELECT * FROM read_json({files})"
df = con.sql(query).df()

In [ ]:
df.head(2)

In [ ]:
agg_df = df.groupby("Model ID", as_index=False).agg(
    avg_total_runtime=("Total_Runtime", "mean"),
    avg_model_runtime=("Model_Runtime", "mean"),
    avg_tokens_generated=("Tokens_Generated", "mean"),
    avg_peak_memory_alloc=("Peak_Memory_Allocated", "mean"),
    avg_peak_memory_reserved=("Peak_Memory_Reserved", "mean"),
    avg_total_frames=("Total_Frames", "mean"),
).sort_values("avg_model_runtime", ascending=False).round(2)
agg_df

In [6]:
from bench_lib.evaluation import visualize_runtime


fig, _ = visualize_runtime(
    df.query("`Model ID`.str.contains('gemma')"), hue="Model ID", plot_colorbar=False
)
fig.savefig("imgs/runtime_scatter_g3_by_model.pdf", bbox_inches="tight")

In [7]:
fig, _ = visualize_runtime(df.query("`Model ID`.str.contains('gemma')"))
fig.savefig("imgs/runtime_scatter_g3_by_frames.pdf", bbox_inches="tight")

In [8]:
fig, _ = visualize_runtime(df.query("`Model ID`.str.contains('Qwen')"), hue="Model ID", plot_colorbar=False)
fig.savefig("imgs/runtime_scatter_qwenvl_by_model.pdf", bbox_inches="tight")

In [9]:
fig, _ = visualize_runtime(df.query("`Model ID`.str.contains('Qwen')"))
fig.savefig("imgs/runtime_scatter_qwenvl_by_frames.pdf", bbox_inches="tight")

In [ ]:
df.query("`Model ID`.str.contains('gemini')")["Total_Runtime"].describe()